

# **Tiki Web Scraping with Selenium**


Build a web-crawler that take in a Tiki URL and return a dataframe 

#Install resources

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
### IMPORTS ###
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [ ]:
###############
### GLOBALS ###
###############

# Header for chromedriver
HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
# Urls
TIKI                = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

#Function to Start and Close Driver

In [ ]:
# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if DRIVER is not None:
        if force_restart:
            DRIVER.close()
        else:
            raise RuntimeError('ERROR: cannot overwrite an active driver. Please close the driver before restarting.')
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

#Function to get info from one product

In [ ]:
# Function to extract product info from the product
def get_product_info_single(product_item):
    d = {'name':'',
         'price':'',
         'product_url':'',
         'image':''}

    # name get name through find_element_by_class_name
    try:
        name_sample = product_item.find_element_by_class_name('name').get_attribute('innerHTML')
        pattern = r'<span>.+'
        name = re.search(pattern, name_sample).group().strip("<///'span>").strip()
        d['name'] = name
    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        d['price'] = product_item.find_element_by_xpath(".//div[contains(@class, 'price-discount__price')]").get_attribute('innerHTML')
    except NoSuchElementException:
        d['price'] = -1
    
    # get link from .get_attribute()
    try:
        product_link     = product_item.get_attribute('href')
        d['product_url'] = product_link
    except NoSuchElementException:
        pass
    
    # get thumbnail by class_name and Tag name and get_attribute()
    try:
        thumbnail = product_item.find_element_by_xpath(".//div[contains(@class, 'thumbnail')]//img[@alt]")
        d['image'] = thumbnail.get_attribute('src')
    except NoSuchElementException:
        pass
    
    return d



#Function to scrape info of all products from a Page URL

In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
    """ Extract info from all products of a specfic page_url on Tiki website
        Args:
            page_url: (string) url of the page to scrape
        Returns:
            data: list of dictionary of products info. If no products shown, return empty list.
    """
    global DRIVER
    data = []
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(5) ## Must have the sleep function

    products_all = DRIVER.find_elements_by_class_name('product-item')
    print(f'Found {len(products_all)} products')

    for product in products_all:
      #Look through the product and get the data
      data.append(get_product_info_single(product))
    return data

In [ ]:
######################
### START SCRAPING ###
######################

num_max_page = int(input('Nhập số lượng trang tối đa cần lấy data: '))


def url_set(): # Cho người dùng chọn ngành hàng và trả về list các trang web trong ngành hàng đó theo số lượng trang web định sẵn
  global num_max_page
  print('-' * 20 + '\nDanh sách ngành hàng:')
  for i in range (len(MAIN_CATEGORIES)):
    print(i+1,'-', MAIN_CATEGORIES[i]['Name']) #in toàn bộ list các ngành hàng

  user_choice = int(input('-' * 20 + '\nMời nhập số thứ tự ngành hàng: '))  # Người dùng nhập số thứ tự ngành hàng để chọn ngành hàng cần lấy data
  while user_choice not in range (1,16):
    print ('Nhập theo số thứ tự ngành hàng: 1, 2, ..., 14, 15')
    user_choice = int(input('Mời nhập số thứ tự ngành hàng: '))

  main_cat_url = MAIN_CATEGORIES[user_choice - 1]['URL'] # lấy link chính của ngành hàng
  main_cat_name = MAIN_CATEGORIES[user_choice - 1]['Name']
  
  print()
  print('-' * 20)
  print(f'{user_choice} - {main_cat_name}')

  url_set = []
  for n in range (num_max_page):
    page_sub_text = r'\1page='+str(n+1)+r'&\2'
    url_set.append(re.sub(r'(https:\/\/tiki.vn\/.*\?)(src.*)',page_sub_text, main_cat_url)) # thêm số trang vào link 'page=(số trang)&'
  return url_set # trả về list các trang web thuộc ngành hàng

close_driver()
start_driver(force_restart=True)

#CODE TO GET DATA
prod_data = []
for page_url in url_set():
  temp_data = get_product_info_from_page(page_url)
  print(temp_data)
  prod_data.extend(temp_data)

close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.to_csv('tiki_products.csv')


Nhập số lượng trang tối đa cần lấy data: 3
--------------------
Danh sách ngành hàng:
1 - Điện Thoại - Máy Tính Bảng
2 - Điện Tử - Điện Lạnh
3 - Phụ Kiện - Thiết Bị Số
4 - Laptop - Thiết bị IT
5 - Máy Ảnh - Quay Phim
6 - Điện Gia Dụng
7 - Nhà Cửa Đời Sống
8 - Hàng Tiêu Dùng - Thực Phẩm
9 - Đồ chơi, Mẹ & Bé
10 - Làm Đẹp - Sức Khỏe
11 - Thể Thao - Dã Ngoại
12 - Xe Máy, Ô tô, Xe Đạp
13 - Hàng quốc tế
14 - Sách, VPP & Quà Tặng
15 - Voucher - Dịch Vụ - Thẻ Cào
--------------------
Mời nhập số thứ tự ngành hàng: 5

--------------------
5 - Máy Ảnh - Quay Phim
Found 64 products
[{'name': 'Camera IP Wifi TP-Link Tapo C200 Full HD 1080P Giám sát An Ninh - Hàng Chính Hãng', 'price': '469.000 ₫', 'product_url': 'https://tka.tiki.vn/pixel?data=djAwMWsU3ZaFe7S21hc_uReJzVB2S3J5bZ-e4thb0EWJ41ESniDpE76EfjIgY47tTyEzJLAKVF9SOWbWtiA5HlNo3cag4QfTQT0DnIBf-gpGwhVyse8kxp5fQw19JVBY5-sWIDSNIbuYWHUfB68r7y60Z6E7ZqsKHavn1lHJBW8ugAtz4l7WEHUe06sZkGC_pNo-fo7eGydFRQ9y3lKijuALVNEP4-oIlUtvYip27tLvd1VSnXVlgSLk49VdINmXyA

In [ ]:
df.head()

,name,price,product_url,image
0,Camera IP Wifi TP-Link Tapo C200 Full HD 1080P...,469.000 ₫,https://tka.tiki.vn/pixel?data=djAwMWsU3ZaFe7S...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,Camera IP Wifi Ezviz C6N 1080p-Hàng Chính Hãng,488.000 ₫,https://tiki.vn/camera-ip-wifi-ezviz-c6n-1080p...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,Camera IP Wifi TP-Link Tapo C200 Full HD 1080P...,469.000 ₫,https://tiki.vn/camera-ip-wifi-tp-link-tapo-c2...,https://salt.tikicdn.com/cache/280x280/ts/prod...
3,Camera chống trộm Xiaomi Mi Home 360° 1080P 20...,689.000 ₫,https://tiki.vn/camera-chong-trom-xiaomi-mi-ho...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,Ống nhòm cao cấp Carson Tracker TZ-821 - Hàng ...,1.321.000 ₫,https://tka.tiki.vn/pixel?data=djAwMQ0Wh__Uw_w...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         174 non-null    object
 1   price        174 non-null    object
 2   product_url  174 non-null    object
 3   image        174 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB
